# Statističke Procene i Procene Greške

## Seminarski rad - SISJ

### Deo 7: Statistička estimacija, regresija i multi-plot mreže

---

## 1. Uvod u statističke procene

**Statistička estimacija** je proces korišćenja podataka iz uzorka za **procenu parametara populacije**. Seaborn pruža napredne alate za vizualizaciju različitih tipova statističkih procena.

### 1.1 Tipovi statističkih procena:

#### **Point estimates (tačkaste procene)**
- Mean (aritmetička sredina)
- Median (medijana)
- Mode (mod)
- Standard deviation (standardna devijacija)

#### **Interval estimates (intervalne procene)**
- Confidence intervals (intervali pouzdanosti)
- Standard error (standardna greška)
- Bootstrap intervals

#### **Distribution estimates (procene distribucija)**
- Kernel density estimation (KDE)
- Histogram density estimation
- Parametric distribution fitting

### 1.2 Procena grešaka:

**Standardna greška** meri **preciznost estimacije**:
- Manja standardna greška = preciznija procena
- Zavisi od veličine uzorka i varijabilnosti
- Koristi se za kreiranje confidence intervala

**Bootstrap metoda**:
- Resampling tehnika za procenu distribucije statistike
- Ne zavisi od pretpostavki o distribuciji
- Robust metoda za procenu grešaka

In [ ]:
# Uvoz potrebnih biblioteka
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
import warnings

# Podešavanje
sns.set_style("whitegrid")
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12, 8)

# Učitavanje podataka
tips = sns.load_dataset('tips')
iris = sns.load_dataset('iris')
flights = sns.load_dataset('flights')
mpg = sns.load_dataset('mpg')

print("=== STATISTICKÉ PROCENE I PROCENE GREŠKE ===")
print(f"Tips dataset: {tips.shape}")
print(f"Iris dataset: {iris.shape}")
print(f"Flights dataset: {flights.shape}")
print(f"MPG dataset: {mpg.shape}")

# Osnovne statistické procene za tips dataset
print("\nOsnovne statističke procene za 'total_bill':")
bill_stats = tips['total_bill'].agg(['mean', 'median', 'std', 'sem'])
print(f"Mean: {bill_stats['mean']:.2f}")
print(f"Median: {bill_stats['median']:.2f}")
print(f"Std: {bill_stats['std']:.2f}")
print(f"Standard Error: {bill_stats['sem']:.2f}")

# 95% confidence interval za mean
n = len(tips['total_bill'])
confidence_level = 0.95
alpha = 1 - confidence_level
t_critical = stats.t.ppf(1 - alpha/2, df=n-1)
margin_of_error = t_critical * bill_stats['sem']

ci_lower = bill_stats['mean'] - margin_of_error
ci_upper = bill_stats['mean'] + margin_of_error

print(f"\n95% Confidence Interval za mean: [{ci_lower:.2f}, {ci_upper:.2f}]")
print(f"Interpretacija: Sa 95% pouzdanošću, populacijska srednja vrednost")
print(f"računa je između {ci_lower:.2f}$ i {ci_upper:.2f}$")

## 2. Vizualizacija statističkih procena

Seaborn omogućava vizualizaciju različitih tipova statističkih procena kroz specijalizovane plot funkcije.

### 2.1 Bar plots sa error bars:

**Bar plot** automatski prikazuje:
- **Point estimate** (obično mean) kao visinu bara
- **Error bars** kao confidence interval
- **Estimator** se može menjati (mean, median, etc.)

### 2.2 Point plots:

**Point plot** prikazuje:
- **Point estimates** kao tačke
- **Error bars** kao linije oko tačke
- **Trend** između kategorija
- **Bolje za poređenje** kategorija

### 2.3 Bootstrap metoda:

Seaborn koristi **bootstrap** za procenu confidence intervala:
- n_boot = 1000 (default broj bootstrap uzoraka)
- ci = 95 (default confidence level)
- seed za reproduktibilnost

In [ ]:
# Vizualizacija statističkih procena
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Vizualizacija Statističkih Procena', fontsize=16)

# 1. Bar plot sa confidence intervals
sns.barplot(data=tips, x='day', y='total_bill', ax=axes[0,0])
axes[0,0].set_title('Bar plot sa CI (Bootstrap)')
axes[0,0].set_ylabel('Total Bill ($)')

# 2. Point plot sa confidence intervals
sns.pointplot(data=tips, x='day', y='total_bill', ax=axes[0,1])
axes[0,1].set_title('Point plot sa CI')
axes[0,1].set_ylabel('Total Bill ($)')

# 3. Bar plot sa različitim estimatorom (median)
sns.barplot(data=tips, x='day', y='total_bill', estimator=np.median, ax=axes[0,2])
axes[0,2].set_title('Bar plot sa Median')
axes[0,2].set_ylabel('Total Bill ($)')

# 4. Poređenje različitih grupa
sns.barplot(data=tips, x='day', y='total_bill', hue='time', ax=axes[1,0])
axes[1,0].set_title('Bar plot sa grupisanjem')
axes[1,0].set_ylabel('Total Bill ($)')

# 5. Point plot sa grupisanjem
sns.pointplot(data=tips, x='day', y='total_bill', hue='time', ax=axes[1,1])
axes[1,1].set_title('Point plot sa grupisanjem')
axes[1,1].set_ylabel('Total Bill ($)')

# 6. Kombinacija različitih statistika
# Kreiranje custom DataFrame sa različitim statistikama
daily_stats = tips.groupby('day')['total_bill'].agg(['mean', 'median', 'std']).reset_index()
x_pos = range(len(daily_stats))
axes[1,2].bar(x_pos, daily_stats['mean'], alpha=0.7, label='Mean', width=0.4)
axes[1,2].bar([x + 0.4 for x in x_pos], daily_stats['median'], alpha=0.7, label='Median', width=0.4)
axes[1,2].set_xticks([x + 0.2 for x in x_pos])
axes[1,2].set_xticklabels(daily_stats['day'])
axes[1,2].set_title('Mean vs Median po danima')
axes[1,2].set_ylabel('Total Bill ($)')
axes[1,2].legend()

plt.tight_layout()
plt.show()

# Analiza statističkih razlika između grupa
print("\n=== STATISTIČKA ANALIZA GRUPA ===")

# ANOVA test za razlike između dana
groups = [tips[tips['day'] == day]['total_bill'] for day in tips['day'].unique()]
f_stat, p_value = stats.f_oneway(*groups)

print(f"ANOVA F-statistika: {f_stat:.3f}")
print(f"P-vrednost: {p_value:.4f}")

if p_value < 0.05:
    print("✅ Postoji statistički značajna razlika između dana (p < 0.05)")
else:
    print("❌ Nema statistički značajne razlike između dana (p >= 0.05)")

# Detaljnije statistike po danima
print("\nDetaljne statistike po danima:")
detailed_stats = tips.groupby('day')['total_bill'].agg([
    'count', 'mean', 'median', 'std', 'sem',
    lambda x: np.percentile(x, 25),  # Q1
    lambda x: np.percentile(x, 75)   # Q3
]).round(2)
detailed_stats.columns = ['Count', 'Mean', 'Median', 'Std', 'SEM', 'Q1', 'Q3']
print(detailed_stats)

## 3. Regresijska analiza

**Regresijska analiza** je metoda za modelovanje odnosa između **zavisne** i **nezavisnih varijabli**. Seaborn pruža različite alate za vizualizaciju regresije.

### 3.1 Tipovi regresije:

#### **Linearna regresija**
- Jednostavan linearni odnos: y = ax + b
- Pretpostavka linearnosti
- Confidence i prediction intervali

#### **Polinomijalna regresija**
- Nelinearni odnosi: y = ax² + bx + c
- Veći stepen polinoma = kompleksniji model
- Pažnja na overfitting

#### **Logistic regresija**
- Za binarnu klasifikaciju
- Output između 0 i 1
- Sigmoid funkcija

### 3.2 Evaluacija regresije:

**R² (koeficijent determinacije)**:
- Procenat varijanse objašnjen modelom (0-100%)
- Viši R² = bolji model

**RMSE (Root Mean Square Error)**:
- Prosečna greška predviđanja
- U istim jedinicama kao y varijabla
- Niži RMSE = bolji model

**Residuals (reziduali)**:
- Razlika između stvarnih i predviđenih vrednosti
- Trebaju biti normalno distribuirani oko 0
- Homoscedastic (konstantna varijansa)

In [ ]:
# Regresijska analiza
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Regresijska Analiza', fontsize=16)

# 1. Linearna regresija
sns.regplot(data=tips, x='total_bill', y='tip', ax=axes[0,0])
axes[0,0].set_title('Linearna regresija')

# 2. Polinomijalna regresija (stepen 2)
sns.regplot(data=tips, x='total_bill', y='tip', order=2, ax=axes[0,1])
axes[0,1].set_title('Polinomijalna regresija (stepen 2)')

# 3. Regresija bez confidence intervala
sns.regplot(data=tips, x='total_bill', y='tip', ci=None, ax=axes[0,2])
axes[0,2].set_title('Regresija bez CI')

# 4. Regresija sa grupisanjem
sns.lmplot(data=tips, x='total_bill', y='tip', hue='time', height=4)
plt.suptitle('Regresija po grupama')
plt.show()

# Kreiranje nove figure za ostatak analiza
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Detaljnija Regresijska Analiza', fontsize=16)

# 5. Residual plot
sns.residplot(data=tips, x='total_bill', y='tip', ax=axes[0,0])
axes[0,0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
axes[0,0].set_title('Residual Plot')

# 6. Poređenje različitih stepena polinoma
x = tips['total_bill'].values
y = tips['tip'].values

# Fitovanje različitih modela
degrees = [1, 2, 3]
colors = ['blue', 'red', 'green']
r2_scores = []

axes[0,1].scatter(x, y, alpha=0.6, color='gray')

for degree, color in zip(degrees, colors):
    # Kreiranje polinomijalnih features
    poly_features = PolynomialFeatures(degree=degree)
    X_poly = poly_features.fit_transform(x.reshape(-1, 1))
    
    # Fitovanje modela
    model = LinearRegression()
    model.fit(X_poly, y)
    
    # Predviđanje
    X_test = np.linspace(x.min(), x.max(), 100).reshape(-1, 1)
    X_test_poly = poly_features.transform(X_test)
    y_pred = model.predict(X_test_poly)
    
    # Plotting
    axes[0,1].plot(X_test, y_pred, color=color, linewidth=2, 
                   label=f'Degree {degree}')
    
    # R² score
    y_pred_train = model.predict(X_poly)
    r2 = r2_score(y, y_pred_train)
    r2_scores.append(r2)

axes[0,1].set_title('Poređenje stepena polinoma')
axes[0,1].legend()
axes[0,1].set_xlabel('Total Bill')
axes[0,1].set_ylabel('Tip')

# 7. R² scores za različite stepene
axes[1,0].bar(degrees, r2_scores, color=colors, alpha=0.7)
axes[1,0].set_title('R² Score po stepenu polinoma')
axes[1,0].set_xlabel('Stepen polinoma')
axes[1,0].set_ylabel('R² Score')
axes[1,0].set_ylim(0, max(r2_scores) + 0.1)

# Dodavanje vrednosti na barove
for i, (degree, r2) in enumerate(zip(degrees, r2_scores)):
    axes[1,0].text(i, r2 + 0.01, f'{r2:.3f}', ha='center', va='bottom')

# 8. Prediction vs Actual
# Koristimo linearni model za demonstraciju
model = LinearRegression()
model.fit(x.reshape(-1, 1), y)
y_pred = model.predict(x.reshape(-1, 1))

axes[1,1].scatter(y, y_pred, alpha=0.6)
# Dodavanje perfektne linije predviđanja
min_val = min(min(y), min(y_pred))
max_val = max(max(y), max(y_pred))
axes[1,1].plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8)
axes[1,1].set_title('Predicted vs Actual')
axes[1,1].set_xlabel('Actual Tip')
axes[1,1].set_ylabel('Predicted Tip')

plt.tight_layout()
plt.show()

# Detaljnija statistička analiza regresije
print("\n=== REGRESIJSKA ANALIZA - REZULTATI ===")

# Linearna regresija sa scipy
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

print(f"Linearna regresija: tip = {slope:.3f} * total_bill + {intercept:.3f}")
print(f"R² koeficijent: {r_value**2:.3f} ({r_value**2*100:.1f}%)")
print(f"P-vrednost: {p_value:.2e}")
print(f"Standardna greška: {std_err:.4f}")

# RMSE calculation
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"RMSE: {rmse:.3f}$")

print(f"\nInterpretacija:")
print(f"• Za svaki dolar povećanja računa, napojnica raste za {slope:.2f}$")
print(f"• Model objašnjava {r_value**2*100:.1f}% varijanse u napojnicama")
print(f"• Prosečna greška predviđanja je {rmse:.2f}$")

if p_value < 0.001:
    print(f"• Regresija je statistički vrlo značajna (p < 0.001)")
elif p_value < 0.01:
    print(f"• Regresija je statistički vrlo značajna (p < 0.01)")
elif p_value < 0.05:
    print(f"• Regresija je statistički značajna (p < 0.05)")
else:
    print(f"• Regresija nije statistički značajna (p >= 0.05)")

## 4. Multi-plot mreže i podešavanja grafikona

**Multi-plot mreže** omogućavaju kreiranje kompleksnih vizualizacija sa **više panel-a** organizovanih u grid strukturi.

### 4.1 FacetGrid:

**FacetGrid** je osnova za kreiranje multi-plot mreža:
- **col**: podela u kolone na osnovu varijable
- **row**: podela u redove na osnovu varijable  
- **hue**: grupisanje unutar svakog panel-a
- **aspect**: kontrola aspect ratio
- **height**: kontrola veličine panel-a

### 4.2 PairGrid:

**PairGrid** za pairwise relationships:
- Matrica scatter plot-ova
- Različiti plot-ovi na dijagonali vs van dijagonale
- Korelacijska analiza između svih varijabli

### 4.3 Podešavanja grafikona:

**Estetska podešavanja**:
- **set_style()**: whitegrid, darkgrid, white, dark, ticks
- **set_context()**: paper, notebook, talk, poster
- **set_palette()**: deep, muted, bright, pastel, dark, colorblind

**Layout podešavanja**:
- **despine()**: uklanjanje ivica
- **tight_layout()**: optimizacija layout-a
- **subplots_adjust()**: ručno podešavanje spacing-a

In [ ]:
# Multi-plot mreže
print("=== MULTI-PLOT MREŽE ===")

# 1. FacetGrid - osnovno
g1 = sns.FacetGrid(tips, col='time', row='smoker', height=4, aspect=1)
g1.map(sns.scatterplot, 'total_bill', 'tip')
g1.add_legend()
plt.suptitle('FacetGrid - Scatter plots po vremenu i pušenju', y=1.02)
plt.show()

# 2. FacetGrid sa različitim plot tipovima
g2 = sns.FacetGrid(tips, col='day', height=4, aspect=0.8)
g2.map(sns.histplot, 'total_bill', kde=True)
plt.suptitle('FacetGrid - Histogrami po danima', y=1.02)
plt.show()

# 3. PairGrid za iris dataset
iris_numeric = iris.select_dtypes(include=[np.number])
g3 = sns.PairGrid(iris, hue='species', height=2.5)
g3.map_diag(sns.histplot)
g3.map_upper(sns.scatterplot)
g3.map_lower(sns.scatterplot)
g3.add_legend()
plt.suptitle('PairGrid - Iris dataset pairwise relationships', y=1.02)
plt.show()

# 4. Kompleksniji FacetGrid sa custom funkcijama
def annotated_scatter(x, y, **kwargs):
    """Custom funkcija za scatter plot sa korelacijom"""
    ax = plt.gca()
    ax.scatter(x, y, **kwargs)
    
    # Računanje korelacije
    r, p = stats.pearsonr(x, y)
    
    # Dodavanje korelacije kao tekst
    ax.text(0.05, 0.95, f'r = {r:.3f}\np = {p:.3f}', 
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

g4 = sns.FacetGrid(tips, col='time', height=5)
g4.map(annotated_scatter, 'total_bill', 'tip', alpha=0.6)
plt.suptitle('Custom FacetGrid sa korelacionim koeficijentima', y=1.02)
plt.show()

print("\n=== PODEŠAVANJA GRAFIKONA ===")

# Demonstracija različitih stilova i konteksta
styles = ['white', 'whitegrid', 'dark', 'darkgrid']
contexts = ['paper', 'notebook', 'talk', 'poster']

# Kreiranje grid-a za stilove
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()
fig.suptitle('Seaborn stilovi', fontsize=16)

for i, style in enumerate(styles):
    with sns.axes_style(style):  # Privremeno menjanje stila
        ax = axes[i]
        sns.scatterplot(data=tips, x='total_bill', y='tip', 
                       hue='time', ax=ax)
        ax.set_title(f'Style: {style}')

plt.tight_layout()
plt.show()

# Kreiranje grid-a za kontekste
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()
fig.suptitle('Seaborn konteksti', fontsize=16)

for i, context in enumerate(contexts):
    with sns.plotting_context(context):  # Privremeno menjanje konteksta
        ax = axes[i]
        sns.barplot(data=tips, x='day', y='total_bill', ax=ax)
        ax.set_title(f'Context: {context}')

plt.tight_layout()
plt.show()

# Demonstracija paleta boja
palettes = ['deep', 'muted', 'bright', 'pastel', 'dark', 'colorblind']
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()
fig.suptitle('Seaborn palete boja', fontsize=16)

for i, palette in enumerate(palettes):
    ax = axes[i]
    with sns.color_palette(palette):
        sns.countplot(data=tips, x='day', hue='time', ax=ax)
    ax.set_title(f'Palette: {palette}')
    ax.legend().set_visible(False)  # Sakrivanje legende za čistoću

plt.tight_layout()
plt.show()

print("\n=== NAPREDNE CUSTOMIZACIJE ===")

# Kreiranje kompleksne multi-plot mreže sa custom podešavanjima
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Glavni scatter plot
ax_main = fig.add_subplot(gs[1:, :-1])
sns.scatterplot(data=tips, x='total_bill', y='tip', 
               hue='time', style='smoker', size='size', 
               sizes=(50, 200), ax=ax_main)
ax_main.set_title('Glavna vizualizacija sa multiple encodings')

# Histogram za x-osu
ax_hist_x = fig.add_subplot(gs[0, :-1], sharex=ax_main)
sns.histplot(data=tips, x='total_bill', hue='time', ax=ax_hist_x, legend=False)
ax_hist_x.set_xlabel('')
ax_hist_x.set_title('Distribucija Total Bill')

# Histogram za y-osu
ax_hist_y = fig.add_subplot(gs[1:, -1], sharey=ax_main)
sns.histplot(data=tips, y='tip', hue='time', ax=ax_hist_y, legend=False)
ax_hist_y.set_ylabel('')
ax_hist_y.set_title('Distribucija Tip', rotation=270, labelpad=20)

# Korelacijska matrica u gornjem desnom uglu
ax_corr = fig.add_subplot(gs[0, -1])
corr_data = tips[['total_bill', 'tip', 'size']].corr()
sns.heatmap(corr_data, annot=True, cmap='coolwarm', center=0, 
            square=True, ax=ax_corr, cbar=False)
ax_corr.set_title('Korelacije')

plt.suptitle('Kompleksna multi-panel vizualizacija', fontsize=16, y=0.98)
plt.show()

print("ZAKLJUČAK - PREDNOSTI MULTI-PLOT MREŽA:")
print("✅ Omogućavaju poređenje između grupa")
print("✅ Efikasno korišćenje prostora")
print("✅ Simultana analiza više aspekata podataka")
print("✅ Flexibilnost u layout-u")
print("✅ Konzistentno styling kroz panel-e")
print("✅ Mogućnost custom funkcija za specifične potrebe")

## 5. Zaključak

U ovom notebook-u smo detaljno istražili **statističke procene, procene grešaka, regresiju i multi-plot mreže** u Seaborn-u.

### Što smo naučili:

#### **Statističke procene:**
- **Point estimates** (mean, median, mode)
- **Interval estimates** (confidence intervals, standard error)
- **Bootstrap metoda** za robust procenu grešaka
- **Vizualizacija** kroz bar plots i point plots

#### **Regresijska analiza:**
- **Linearna regresija** za jednostavne odnose
- **Polinomijalna regresija** za nelinearne odnose
- **Evaluacija modela** kroz R², RMSE, residuals
- **Vizualizacija** kroz regplot, lmplot, residplot

#### **Multi-plot mreže:**
- **FacetGrid** za grupne analize
- **PairGrid** za pairwise relationships
- **Custom funkcije** za specijalizovane analize
- **Styling** kroz stilove, kontekste i palete

### Ključne prednosti:

1. **Statistička rigoroznost** - automatska procena grešaka
2. **Vizualna jasnoća** - intuitivni prikaz kompleksnih analiza
3. **Fleksibilnost** - prilagođavanje različitim tipovima podataka
4. **Skalabilnost** - od jednostavnih do kompleksnih analiza
5. **Integracija** - seamless rad sa pandas i scipy

### Praktične aplikacije:

- **Exploratory Data Analysis (EDA)** - razumevanje podataka
- **Hipoteza testiranje** - statistička validacija
- **Modelovanje** - kreiranje i evaluacija modela
- **Reporting** - profesionalni prikaz rezultata
- **Komunikacija** - objašnjavanje nalaza stakeholderima

### Sledeći koraci:

Ovaj notebook zaključuje naš detaljni pregled Seaborn biblioteke. Sledeći korak je **klasifikacija slika** gde ćemo demonstrirati algoritam van Seaborn ekosistema, ali sa vizualizacijom rezultata kroz Seaborn.

---

**Napomena**: Ovaj notebook pokriva napredne statističke tehnike. Za dublje razumevanje, preporučujemo proučavanje literature iz oblasti statistike i machine learning-a.